In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
# from datasets import load_metric
# !pip install evaluate
# 
from sklearn.model_selection import train_test_split
import csv
from optuna import Trial
from typing import Dict, Union, Any
import evaluate

In [2]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/multi-lang-exp/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'A':0, 'C':1, 'P':2}
df = df.rename(columns={'maintenance_type':'labels','msgs':'text','commit_diff':'diffs'})
df = df.replace({"labels": label2id})
# # print(df)
df
# test_sample = df.sample(n=3, random_state=1)

/tmp/ipykernel_1619/1422861430.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"labels": label2id})


,language,commit,commit_url,labels,user,repo,feature,diffs,text
0,C++,be937a3290223f926fe50684f1344569a573ed4b,https://github.com/tensorflow/tensorflow/commi...,1,tensorflow,tensorflow,"[1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 6, ...",diff --git a/tensorflow/compiler/mlir/lite/fla...,Fixing per axis quantization bug in flatbuffer...
1,C++,c800d2e36954edddcb83aa1df7f623f2780c7780,https://github.com/tensorflow/tensorflow/commi...,0,tensorflow,tensorflow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/tensorflow/python/tools/saved_mod...,Add support for python expression as SavedMode...
2,C++,469e56eeff17eb857c95e935ca7b49723c43470e,https://github.com/tensorflow/tensorflow/commi...,2,tensorflow,tensorflow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/.github/ISSUE_TEMPLATE/00-bug-iss...,Fix GitHub issue templates. Actual fix for #36...
3,C++,24f5d302c0906f3377f4d6bcc2735d943ce13e3d,https://github.com/tensorflow/tensorflow/commi...,0,tensorflow,tensorflow,"[3, 37, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/tensorflow/compiler/xla/pjrt/dist...,"Add new PjRT distributed APIs (KeyValueDelete,..."
4,C++,1effdb7b959503596b07f7f3e74618ab63f9e5e5,https://github.com/tensorflow/tensorflow/commi...,2,tensorflow,tensorflow,"[1, 0, 12, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/tensorflow/core/common_runtime/ea...,Remove IsLocalDevice() since it's no longer us...
...,...,...,...,...,...,...,...,...,...
1576,Java,e4bbfd537a63a52a6094491504fc7b0bb54bcb0e,,1,kiegroup,drools,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-persistence-jpa/src/main/j...,fixing reflection constructor to use int inste...
1577,Java,75b81dcbd26a11d5b370ea5bdf102c2d62ebd0c3,,1,kiegroup,drools,"[2, 6, 2, 3, 0, 0, 0, 1, 7, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-compiler/src/main/java/org...,Fix OutOfBoundException on- MemoryFileSystem.e...
1578,Java,6bc9677bbe919cd5bcadd8af2f5b0c838757a4ce,,1,kiegroup,drools,"[1, 1, 5, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/drools-core/src/main/java/org/dro...,[DROOLS-293] fix ObjectTypeNode id creation an...
1579,Java,48701edad8513b27acec7216581e64637157c86a,,0,kiegroup,drools,"[2, 24, 2, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/drools-core/src/main/java/org/dro...,DROOLS-515 Kie-Camel is broken after Camel Upd...


In [3]:
# from datasets import Dataset, load_metric

In [4]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [5]:
len(train)

1106

In [6]:
train['labels'].value_counts()

labels
2    691
1    242
0    173
Name: count, dtype: int64

In [7]:
test['labels'].value_counts()

labels
2    155
1     51
0     32
Name: count, dtype: int64

In [8]:
# id2label = {0: "Adaptive", 1: "POSITIVE"}
# label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "/root/autodl-tmp/CommitFit/models/codebert-base", num_labels=3
)
tokenizer = AutoTokenizer.from_pretrained('/root/autodl-tmp/CommitFit/models/codebert-base')

Some weights of the model checkpoint at /root/autodl-tmp/CommitFit/models/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/CommitFit/models/codebert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probab

In [10]:
training_args = TrainingArguments(
    output_dir="./my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none"
)

In [11]:
encoded_train = tokenizer(train['diffs'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_test = tokenizer(test['diffs'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_val = tokenizer(val['diffs'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)

In [12]:
class CommitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [13]:
train_dataset = CommitDataset(encoded_train, list(train['labels']))
test_dataset = CommitDataset(encoded_test, list(test['labels']))
val_dataset = CommitDataset(encoded_val, list(val['labels']))

In [14]:
from sklearn import  metrics
def compute_metrics(eval_pred):
    accuracy = evaluate.load('../../evaluate/metrics/accuracy')
    precision =  evaluate.load("../../evaluate/metrics/precision")
    recall =  evaluate.load("../../evaluate/metrics/recall")
    f1 =  evaluate.load("../../evaluate/metrics/f1")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy.compute(predictions=predictions, references=labels)
    precision = precision.compute(predictions=predictions, references=labels, average='macro')
    recall = recall.compute(predictions=predictions, references=labels,average='macro')
    f1 = f1.compute(predictions=predictions, references=labels,average='macro')
    # accuracy_score = metrics.accuracy_score(labels,predictions)
    # precision_score = metrics.precision_score(labels,predictions, average='weighted')
    # recall_score = metrics.recall_score(labels,predictions,average='weighted')
    # f1_score = metrics.f1_score(labels,predictions,average='weighted')
    # 混淆矩阵
    cm = metrics.confusion_matrix(labels, predictions)
    print("Confusion Matrix:\n", cm)
    print(metrics.classification_report(labels, predictions, digits=4))
    # return {"classification_report": classification_report}
    # return {"precision": precision_score,"recall": recall_score, "f1":f1_score, "accuracy": accuracy_score}
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_1619/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.827015,{'precision': 0.3700729095465938},{'recall': 0.38690753690753693},{'f1': 0.3541666666666667},{'accuracy': 0.6244725738396625}
2,No log,0.784668,{'precision': 0.5373230373230373},{'recall': 0.5014854426619132},{'f1': 0.5094730244065611},{'accuracy': 0.6286919831223629}
3,No log,0.794112,{'precision': 0.5251484753934234},{'recall': 0.5210521504639152},{'f1': 0.520749845987481},{'accuracy': 0.6075949367088608}
4,No log,0.954693,{'precision': 0.5407407407407407},{'recall': 0.4772795831619361},{'f1': 0.487627941599418},{'accuracy': 0.6371308016877637}
5,No log,0.953925,{'precision': 0.5255584693084693},{'recall': 0.5215960510078157},{'f1': 0.5229599030002255},{'accuracy': 0.620253164556962}


Confusion Matrix:
 [[  0   7  27]
 [  0  13  47]
 [  0   8 135]]
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        34
           1     0.4643    0.2167    0.2955        60
           2     0.6459    0.9441    0.7670       143

    accuracy                         0.6245       237
   macro avg     0.3701    0.3869    0.3542       237
weighted avg     0.5073    0.6245    0.5376       237



/root/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/miniconda3/lib/python3.10

Confusion Matrix:
 [[ 12   6  16]
 [  5  20  35]
 [ 15  11 117]]
              precision    recall  f1-score   support

           0     0.3750    0.3529    0.3636        34
           1     0.5405    0.3333    0.4124        60
           2     0.6964    0.8182    0.7524       143

    accuracy                         0.6287       237
   macro avg     0.5373    0.5015    0.5095       237
weighted avg     0.6109    0.6287    0.6106       237



...

/tmp/ipykernel_1619/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Confusion Matrix:
 [[  9   4  21]
 [  7  18  35]
 [ 11   8 124]]
              precision    recall  f1-score   support

           0     0.3333    0.2647    0.2951        34
           1     0.6000    0.3000    0.4000        60
           2     0.6889    0.8671    0.7678       143

    accuracy                         0.6371       237
   macro avg     0.5407    0.4773    0.4876       237
weighted avg     0.6154    0.6371    0.6069       237



/tmp/ipykernel_1619/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Confusion Matrix:
 [[ 11  10  13]
 [  6  30  24]
 [ 13  24 106]]
              precision    recall  f1-score   support

           0     0.3667    0.3235    0.3438        34
           1     0.4688    0.5000    0.4839        60
           2     0.7413    0.7413    0.7413       143

    accuracy                         0.6203       237
   macro avg     0.5256    0.5216    0.5230       237
weighted avg     0.6185    0.6203    0.6191       237



TrainOutput(global_step=350, training_loss=0.691711164202009, metrics={'train_runtime': 86.2308, 'train_samples_per_second': 128.26, 'train_steps_per_second': 8.118, 'total_flos': 1455017200035840.0, 'train_loss': 0.691711164202009, 'epoch': 5.0})

In [16]:
fewshot_metrics = trainer.evaluate(eval_dataset=test_dataset)
fewshot_metrics

/tmp/ipykernel_1619/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Confusion Matrix:
 [[ 10   5  17]
 [ 10  13  28]
 [ 19   9 127]]
              precision    recall  f1-score   support

           0     0.2564    0.3125    0.2817        32
           1     0.4815    0.2549    0.3333        51
           2     0.7384    0.8194    0.7768       155

    accuracy                         0.6303       238
   macro avg     0.4921    0.4623    0.4639       238
weighted avg     0.6185    0.6303    0.6152       238



{'eval_loss': 0.7997500896453857,
 'eval_precision': {'precision': 0.4920879436383312},
 'eval_recall': {'recall': 0.46225226649799706},
 'eval_f1': {'f1': 0.4639272946547788},
 'eval_accuracy': {'accuracy': 0.6302521008403361},
 'eval_runtime': 1.1564,
 'eval_samples_per_second': 205.808,
 'eval_steps_per_second': 12.971,
 'epoch': 5.0}

In [17]:
#发送多种类型的邮件
from email.mime.multipart import MIMEMultipart
import smtplib

from email.mime.text import MIMEText
msg_from = '915803745@qq.com'  # 发送方邮箱
passwd = 'vcuosuurrgkfbdai'   #就是上面的授权码
 
# to= ['g.zhang@gotion.com', 'j.tong@gotion.com'] #接受方邮箱
to= ['j.tong@gotion.com'] #接受方邮箱
#设置邮件内容
#MIMEMultipart类可以放任何内容
msg = MIMEMultipart()
conntent=f"{fewshot_metrics}"
#把内容加进去
msg.attach(MIMEText(conntent,'plain','utf-8'))
 
#设置邮件主题
msg['Subject']="强化学习模型训练完毕"
 
#发送方信息
msg['From']=msg_from
 
#开始发送
 
#通过SSL方式发送，服务器地址和端口
s = smtplib.SMTP_SSL("smtp.qq.com", 465)
# 登录邮箱
s.login(msg_from, passwd)
#开始发送
s.sendmail(msg_from,to,msg.as_string())
print("强化学习模型训练完毕")

强化学习模型训练完毕
